In [1]:
import mpmath as mpmath
import mpmath as mp
mp.mp.dps = 30 #higher precision
from post_processing import *
import pandas as pd
import os

In [2]:
def write_to_pickle(
    df, boundary_type, intervals, iterations,
    overlap, walkers, alpha, beta, exchange, probabilities
    ):

    path = f"../results_curves/{boundary_type}/X_{X}_Y_{Y}/prob_{probabilities:.6f}/free_energies_intervals_{intervals}_iterations_{iterations}_overlap_{overlap:.6f}_walkers_{walkers}_alpha_{alpha:.6f}_beta_{beta:.10f}_exchange_offset_{exchange}.pickle"

    if os.path.exists(path):
        print(f"File {path} already exists, checking duplicates and update dataframe")
        df_orig = pd.read_pickle(path)

        df_combined = pd.concat([df_orig, df])

        df_result = df_combined.drop_duplicates(subset=['histogram_seed', 'error'], keep='first')

        df_result.to_pickle(path)
    else:
        print(f"File {path} does not exist, write to disk")
        os.makedirs(os.path.dirname(path), exist_ok=True)
        df.to_pickle(path)

In [3]:
def rescale_log_g(row):
    log_g_list = row["log_g"]
    X = row["X"]
    Y = row["Y"]

    offset = log_sum_exp(log_g_list)

    rescaled_log_g_list = [
        res + mp.log(2)*X*Y - offset for res in log_g_list
    ]

    return rescaled_log_g_list

In [4]:
def perform_post_processing(
    probabilities, X, Y, boundary_type, intervals,
    iterations, overlap, walkers, alpha, beta, exchange,
    errors = {"I", "X", "Y", "Z"}, low_temp=1e-20, high_temp=1e20
    ):

        batch_results = []

        for error in ["I", "X", "Y", "Z"]:
            filename = f"../results/{boundary_type}/prob_{probabilities:.6f}/X_{X}_Y_{Y}/error_class_{error}/StitchedHistogram_intervals_{intervals}_iterations_{iterations}_overlap_{overlap:.6f}_walkers_{walkers}_alpha_{alpha:.6f}_beta_{beta:.10f}_exchange_offset{exchange}.txt"

            data = read_results_file(filename)
            if data:
                process_data(data, batch_results, probabilities, X, Y, error)

        df = pd.DataFrame(batch_results)

        # Filter based on errors
        df_filtered = df.groupby('histogram_seed').filter(
            lambda x: set(x['error']) == errors and len(x) == len(errors)
        )

        df_filtered['rescaled_log_g'] = df_filtered.apply(rescale_log_g, axis=1)

        df_sorted = df_filtered.sort_values(by=["histogram_seed", "error"])

        nish_temp = 1/(mp.log((1-probabilities)/probabilities)/2)

        df_sorted[f"free_energy_low_temp"] = df_sorted.apply(lambda row: free_energy(row['E'], row['rescaled_log_g'], low_temp)/(-low_temp), axis=1)
        df_sorted[f"free_energy_nish_temp"] = df_sorted.apply(lambda row: free_energy(row['E'], row['rescaled_log_g'], nish_temp)/(-nish_temp), axis=1)
        df_sorted[f"free_energy_high_temp"] = df_sorted.apply(lambda row: free_energy(row['E'], row['rescaled_log_g'], high_temp)/(-high_temp), axis=1)

        write_to_pickle(
            df_sorted, boundary_type, intervals, iterations,
            overlap, walkers, alpha, beta, exchange, probabilities
        )

In [5]:
boundary_type = "periodic"
intervals = 10
iterations = 1000
overlap = 0.25
walkers = 8
alpha = 0.8
beta = 1e-8
exchange = 20

probabilities = [0.1, 0.11, 0.12, 0.13]
X = 6
Y = 6
errors = ["I", "X", "Y", "Z"]

In [6]:
for p in probabilities:

    perform_post_processing(
        p, X, Y, boundary_type, intervals,
        iterations, overlap, walkers, alpha, beta, exchange
    )

File ../results_curves/periodic/X_6_Y_6/prob_0.100000/free_energies_intervals_10_iterations_1000_overlap_0.250000_walkers_8_alpha_0.800000_beta_0.0000000100_exchange_offset_20.pickle does not exist, write to disk
File ../results_curves/periodic/X_6_Y_6/prob_0.110000/free_energies_intervals_10_iterations_1000_overlap_0.250000_walkers_8_alpha_0.800000_beta_0.0000000100_exchange_offset_20.pickle does not exist, write to disk
File ../results_curves/periodic/X_6_Y_6/prob_0.120000/free_energies_intervals_10_iterations_1000_overlap_0.250000_walkers_8_alpha_0.800000_beta_0.0000000100_exchange_offset_20.pickle does not exist, write to disk
File ../results_curves/periodic/X_6_Y_6/prob_0.130000/free_energies_intervals_10_iterations_1000_overlap_0.250000_walkers_8_alpha_0.800000_beta_0.0000000100_exchange_offset_20.pickle does not exist, write to disk
